In [5]:
import cv2                      
import numpy as np              

# =========================
# Spatial-domain functions
# =========================

def NoisRedeuction(img, Kernalsize):
    # Apply Gaussian blur to reduce noise; Kernalsize is the blur kernel size
    blurred = cv2.GaussianBlur(img, Kernalsize, 0)  # sigmaX=0 lets OpenCV choose it
    return blurred  # Return the blurred image
    
def Gradient(img):
    # Compute gradients in x and y directions using Sobel operator
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)  # Horizontal gradient
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)  # Vertical gradient

    # Compute gradient magnitude (edge strength)
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    # Compute gradient direction (edge orientation)
    grad_direction = np.arctan2(grad_y, grad_x)

    # Normalize gradient magnitude to 0–255 for display
    grad_magnitude_disp = cv2.normalize(grad_magnitude, None, 0, 255, cv2.NORM_MINMAX)
    grad_magnitude_disp = grad_magnitude_disp.astype(np.uint8)

    # Normalize gradient direction to 0–255 for display 
    grad_direction_disp = cv2.normalize(grad_direction, None, 0, 255, cv2.NORM_MINMAX)
    grad_direction_disp = grad_direction_disp.astype(np.uint8)

    # Return magnitude (for edges), direction, and display version of magnitude
    return grad_magnitude, grad_direction, grad_magnitude_disp    

def non_maximum_suppression(grad_mag, grad_angle):
    # Get image height and width
    M, N = grad_mag.shape
    # Create an empty array for suppressed magnitude
    Z = np.zeros((M, N), dtype=np.float32)
    # Convert gradient direction from radians to degrees
    angle = grad_angle * 180. / np.pi
    # Make all angles in range [0, 180)
    angle[angle < 0] += 180    

    # Loop over interior pixels (ignore border pixels)
    for i in range(1, M - 1):
        for j in range(1, N - 1):
            q = 255  # Neighbor 1 magnitude placeholder
            r = 255  # Neighbor 2 magnitude placeholder

            # 0 degrees (horizontal edges)
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                q, r = grad_mag[i, j + 1], grad_mag[i, j - 1]

            # 45 degrees (diagonal)
            elif (22.5 <= angle[i, j] < 67.5):
                q, r = grad_mag[i + 1, j - 1], grad_mag[i - 1, j + 1]

            # 90 degrees (vertical edges)
            elif (67.5 <= angle[i, j] < 112.5):
                q, r = grad_mag[i + 1, j], grad_mag[i - 1, j]

            # 135 degrees (other diagonal)
            elif (112.5 <= angle[i, j] < 157.5):
                q, r = grad_mag[i - 1, j - 1], grad_mag[i + 1, j + 1]

            # Keep the pixel if it is greater than both neighbors (local maximum)
            if grad_mag[i, j] >= q and grad_mag[i, j] >= r:
                Z[i, j] = grad_mag[i, j]
            else:
                Z[i, j] = 0  # Otherwise suppress it (set to zero)

    return Z  # Return thinned edges

def double_threshold(image, low_threshold, high_threshold):
    # Define intensity for strong and weak edges
    strong = 255
    weak = 75

    # Initialize output images for strong and weak edges
    strong_edges = np.zeros_like(image, dtype=np.uint8)
    weak_edges = np.zeros_like(image, dtype=np.uint8)

    # Mark strong edges (above high threshold)
    strong_edges[image >= high_threshold] = strong
    # Mark weak edges (between low and high thresholds)
    weak_edges[(image >= low_threshold) & (image < high_threshold)] = weak

    # Return strong and weak edge maps
    return strong_edges, weak_edges
    
def edge_tracking_by_hysteresis(strong_edges, weak_edges):
    # Get image size
    M, N = strong_edges.shape
    # Start from strong edges
    result = np.copy(strong_edges)
    weak = 75
    strong = 255

    # Check each pixel except the border
    for i in range(1, M - 1):
        for j in range(1, N - 1):
            # If current pixel is weak
            if weak_edges[i, j] == weak:
                # If any neighbor is strong, promote to strong edge
                if (
                    (result[i+1, j-1] == strong) or
                    (result[i+1, j]   == strong) or
                    (result[i+1, j+1] == strong) or
                    (result[i, j-1]   == strong) or
                    (result[i, j+1]   == strong) or
                    (result[i-1, j-1] == strong) or
                    (result[i-1, j]   == strong) or
                    (result[i-1, j+1] == strong)
                ):
                    result[i, j] = strong
                else:
                    # Otherwise discard the weak edge
                    result[i, j] = 0
    # Return final edge map after hysteresis
    return result

def auto_threshold(image):
    """Automatically choose low and high thresholds based on image stats."""
    mean_val = np.mean(image)  # Compute average brightness
    std_val = np.std(image)    # Compute contrast (standard deviation)
    
    # Low threshold: mean minus half the standard deviation, but not below 5
    low_threshold = int(max(5, mean_val - 0.5 * std_val))
    # High threshold: mean plus 1.5 times standard deviation, but not below 30
    high_threshold = int(max(30, mean_val + 1.5 * std_val))
    # Clamp low threshold into [5, 150]
    low_threshold = np.clip(low_threshold, 5, 150)
    # Clamp high threshold into [20, 255]
    high_threshold = np.clip(high_threshold, 20, 255)
    return low_threshold, high_threshold  # Return chosen thresholds

def Canny(image, kernel_size, low_thresh=None, high_thresh=None, auto_thresh=True):
    # Step 1: Remove noise with Gaussian blur
    img = NoisRedeuction(image, kernel_size)
    # Step 2: Compute gradient magnitude and direction
    mag, ang, disp = Gradient(img)

    # Normalize gradient magnitude to 0–255 and convert to uint8
    mag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    mag = mag.astype(np.uint8)

    # Step 3: Apply non-maximum suppression to thin edges
    nms = non_maximum_suppression(mag, ang)
    # Normalize suppressed result and convert to uint8
    nms = cv2.normalize(nms, None, 0, 255, cv2.NORM_MINMAX)
    nms = nms.astype(np.uint8)
    
    # Step 4: Choose thresholds (auto or manual)
    if auto_thresh or low_thresh is None or high_thresh is None:
        low_threshold, high_threshold = auto_threshold(nms)
    else:
        low_threshold, high_threshold = low_thresh, high_thresh
    
    # Step 5: Apply double threshold to classify strong/weak edges
    strong_edges, weak_edges = double_threshold(nms, low_threshold, high_threshold)
    # Step 6: Apply hysteresis to link edges
    img = edge_tracking_by_hysteresis(strong_edges, weak_edges)
    img = img.astype(np.uint8)  # Ensure uint8 type for display
    # Return final edges and the thresholds used
    return img, low_threshold, high_threshold

# =========================
# Frequency-domain helpers
# =========================

def to_freq(img):
    """Convert gray image to shifted frequency domain representation."""
    f = np.fft.fft2(img)        # 2D FFT
    f_shift = np.fft.fftshift(f)  # Shift zero-frequency to center
    return f_shift

def from_freq(f_shift):
    """Convert shifted frequency domain back to spatial gray image."""
    f_ishift = np.fft.ifftshift(f_shift)  # Undo shift
    img_back = np.fft.ifft2(f_ishift)     # Inverse FFT
    img_back = np.abs(img_back)           # Take magnitude (real image)
    # Normalize to 0–255 and convert to uint8
    img_back = cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX)
    return img_back.astype(np.uint8)

def make_filter_mask(shape, ftype='ideal', mode='low', D0=30, n=2, W=20):
    """
    Create a frequency-domain filter mask.

    shape: image size (rows, cols)
    ftype: filter type 'ideal' | 'gaussian' | 'butterworth'
    mode : 'low' | 'high' | 'bandreject' | 'bandpass'
    D0   : cutoff or center radius
    n    : Butterworth order
    W    : band width (for band filters)
    """
    rows, cols = shape          # Get image shape
    crow, ccol = rows // 2, cols // 2  # Center of the spectrum

    # Build distance matrix D(u,v) from center
    u = np.arange(rows)
    v = np.arange(cols)
    V, U = np.meshgrid(v, u)
    D = np.sqrt((U - crow) ** 2 + (V - ccol) ** 2)

    # Base low-pass mask (for 'low' or 'high')
    if mode in ['low', 'high']:
        if ftype == 'ideal':
            # Ideal low-pass: inside radius D0 is 1, outside is 0
            H = np.zeros_like(D, dtype=np.float32)
            H[D <= D0] = 1.0
        elif ftype == 'gaussian':
            # Gaussian low-pass: smooth decay with distance
            H = np.exp(-(D ** 2) / (2 * (D0 ** 2)))
        elif ftype == 'butterworth':
            # Butterworth low-pass: controlled by order n
            D_safe = np.where(D == 0, 1e-6, D)  # Avoid divide by zero
            H = 1.0 / (1.0 + (D_safe / D0) ** (2 * n))
        else:
            # Invalid filter type
            raise ValueError("Unknown ftype")

        # Convert low-pass to high-pass by subtracting from 1
        if mode == 'high':
            H = 1.0 - H

    # Bandreject / bandpass masks
    else:
        D_safe = np.where(D == 0, 1e-6, D)  # Avoid divide by zero
        if ftype == 'ideal':
            # Start with all-pass (ones)
            H = np.ones_like(D, dtype=np.float32)
            # Define inner and outer radii of the band
            D_low = D0 - W / 2.0
            D_high = D0 + W / 2.0
            # Set values in the band to 0 (reject band)
            H[(D >= D_low) & (D <= D_high)] = 0.0
        elif ftype == 'gaussian':
            # Gaussian bandreject (smooth dip around D0)
            H = 1.0 - np.exp(-(((D_safe**2 - D0**2) / (D_safe * W + 1e-6)) ** 2))
        elif ftype == 'butterworth':
            # Butterworth bandreject
            H = 1.0 / (1.0 + ((D_safe * W) / (D_safe**2 - D0**2 + 1e-6)) ** (2 * n))
        else:
            raise ValueError("Unknown ftype")

        # Convert bandreject to bandpass by inverting
        if mode == 'bandpass':
            H = 1.0 - H

    # Return the frequency mask
    return H

def freq_filter(img, ftype='ideal', mode='low', D0=30, n=2, W=20):
    """
    Apply a frequency-domain filter to a grayscale image.

    img  : gray uint8 image
    ftype: 'ideal' | 'gaussian' | 'butterworth'
    mode : 'low' | 'high' | 'bandreject' | 'bandpass'
    D0   : cutoff / center radius in pixels
    n    : Butterworth filter order
    W    : band width for band filters
    """
    # Convert image to frequency domain
    f_shift = to_freq(img)
    # Build a filter mask with the same shape as image
    H = make_filter_mask(img.shape, ftype=ftype, mode=mode, D0=D0, n=n, W=W)
    # Apply mask by element-wise multiplication
    f_filtered = f_shift * H
    # Transform back to spatial domain
    img_back = from_freq(f_filtered)
    # Return filtered image
    return img_back

# =========================
# Main application / UI
# =========================

# Open default camera (index 0)
cap = cv2.VideoCapture(0)
# Initial kernel size for blur and Canny
kernel_size = (5, 5)
# Initial Canny thresholds (used in manual mode)
low_threshold = 100
high_threshold = 200
# Current display mode: 'normal', 'canny', 'blur', 'sobel', or frequency modes
mode = 'normal'
# Flags to control whether to draw threshold info text
show_thresh_info = False
# Flag to control whether to draw Sobel info text
show_sobel_info = False   # New flag to show Sobel info only in Sobel mode
# Flag to select automatic Canny thresholds
auto_mode = True

# Frequency filter settings
filter_type = 'ideal'       # Current filter type: 'ideal' | 'gaussian' | 'butterworth'
filter_mode = 'low'         # Not used directly (mode is mapped from 'lpf','hpf',...)
D0 = 30                     # Cutoff or center frequency radius
butter_n = 2                # Butterworth filter order
band_W = 40                 # Band width for band filters

# Print keyboard control instructions
print("=== CONTROLS ===")
print("c/b/s/n  - Canny/Blur/Sobel/Normal")
print("l/h      - Low‑pass / High‑pass (frequency)")
print("r/p      - BandReject / BandPass (frequency, uses W)")
print("i/g/t    - Ideal / Gaussian / Butterworth filter type")
print("[/]      - D0 (cutoff / band center) -/+")
print("w/W      - Band width W -/+")
print("1/2/3    - Butterworth order")
print("+/-      - Kernel size for blur/Canny")
print("A        - Toggle AUTO ↔ MANUAL thresholds (Canny)")
print("</>      - Adjust LOW threshold (MANUAL only)")
print(" u/d     - Adjust HIGH threshold ±20 (MANUAL only)")
print("q        - Quit")
print("================")

# Main loop: process frames until user quits
while True:
    # Read a frame from the camera
    ret, frame = cap.read()
    # If frame not captured, skip this iteration
    if not ret:
        continue

    # Convert frame from BGR color to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # --------- Spatial modes ---------
    if mode == 'canny':
        # Apply custom Canny edge detector
        edges, low_t, high_t = Canny(
            gray,
            kernel_size, 
            low_thresh=low_threshold if not auto_mode else None,
            high_thresh=high_threshold if not auto_mode else None,
            auto_thresh=auto_mode
        )
        # Update thresholds (Auto mode returns new values)
        low_threshold = low_t
        high_threshold = high_t
        # Convert edges (gray) to BGR for display overlay
        display = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        show_thresh_info = True   # Show kernel/threshold info
        show_sobel_info = False   # Hide Sobel info

    elif mode == 'blur':
        # Apply Gaussian blur to grayscale image
        blurred = NoisRedeuction(gray, kernel_size)
        # Convert blurred image to BGR for display
        display = cv2.cvtColor(blurred, cv2.COLOR_GRAY2BGR)
        show_thresh_info = True   # Show kernel/threshold info
        show_sobel_info = False   # Hide Sobel info

    elif mode == "sobel":
        # Compute gradient magnitude and direction
        mag, ang, edges = Gradient(gray)
        # Convert gradient magnitude image to BGR for display
        display = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        show_thresh_info = False  # Do not show threshold info
        show_sobel_info = True    # Show Sobel informational text

    # --------- Frequency modes ---------
    elif mode in ['lpf', 'hpf', 'brf', 'bpf']:
        # Map mode to frequency filter mode string
        if mode == 'lpf':
            fm = 'low'
        elif mode == 'hpf':
            fm = 'high'
        elif mode == 'brf':
            fm = 'bandreject'
        else:
            fm = 'bandpass'

        # Apply selected frequency filter to gray image
        filtered = freq_filter(gray,
                               ftype=filter_type,
                               mode=fm,
                               D0=D0,
                               n=butter_n,
                               W=band_W)
        # Convert filtered result to BGR for display
        display = cv2.cvtColor(filtered, cv2.COLOR_GRAY2BGR)
        show_thresh_info = False  # No Canny/blur info in frequency mode
        show_sobel_info = False   # No Sobel info either

    else:
        # Normal mode: show original camera frame
        display = frame
        show_thresh_info = False
        show_sobel_info = False

    # ------------- Overlays -------------
    # Overlay kernel and threshold information if needed
    if show_thresh_info:
        info_y = 30  # Initial text y-position
        kernel_text = f"Kernel: {kernel_size[0]}x{kernel_size[1]} | +/-"
        cv2.putText(display, kernel_text, (10, info_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        info_y += 30  # Move down for next line of text
        if auto_mode:
            # Show current thresholds in AUTO mode
            thresh_text = f"AUTO: L{low_threshold} H{high_threshold} | A=toggle"
        else:
            # Show current thresholds and manual control keys
            thresh_text = f"MANUAL: L{low_threshold} H{high_threshold} | </> L | u/d H"
        cv2.putText(display, thresh_text, (10, info_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Overlay Sobel info if in Sobel mode
    if show_sobel_info:
        info_y = 30
        sobel_text = "SOBEL: gradient magnitude, ksize=3"
        cv2.putText(display, sobel_text, (10, info_y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Overlay frequency filter info when in frequency mode
    if mode in ['lpf', 'hpf', 'brf', 'bpf']:
        # Map mode to text label
        mode_txt = {'lpf': 'LOW‑PASS', 'hpf': 'HIGH‑PASS',
                    'brf': 'BAND‑REJECT', 'bpf': 'BAND‑PASS'}[mode]
        # Build info text with current filter parameters
        txt = f"{mode_txt} {filter_type.upper()} D0={D0} n={butter_n} W={band_W}"
        cv2.putText(display, txt, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Show the current processed frame in a window
    cv2.imshow("Canny / Blur / Sobel / LPF / HPF / BRF / BPF", display)

    # Wait for key press for 1 ms and get key code
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        # Quit application if 'q' is pressed
        break

    # --------- Mode switching (spatial) ---------
    elif key == ord('c'):
        # Toggle between Canny and normal mode
        mode = "canny" if mode != "canny" else "normal"
    elif key == ord('b'):
        # Toggle between Blur and normal mode
        mode = "blur" if mode != "blur" else "normal"
    elif key == ord('s'):
        # Toggle between Sobel and normal mode
        mode = "sobel" if mode != "sobel" else "normal"
    elif key == ord('n'):
        # Switch to normal mode explicitly
        mode = 'normal'

    # --------- Mode switching (frequency) ---------
    elif key == ord('l'):
        # Switch to low-pass frequency mode
        mode = 'lpf'
    elif key == ord('h'):
        # Switch to high-pass frequency mode
        mode = 'hpf'
    elif key == ord('r'):
        # Switch to band-reject frequency mode
        mode = 'brf'
    elif key == ord('p'):
        # Switch to band-pass frequency mode
        mode = 'bpf'

    # --------- Filter type selection ---------
    elif key == ord('i'):
        # Use ideal filter
        filter_type = 'ideal'
    elif key == ord('g'):
        # Use Gaussian filter
        filter_type = 'gaussian'
    elif key == ord('t'):
        # Use Butterworth filter
        filter_type = 'butterworth'

    # --------- Adjust D0 (cutoff or band center) ---------
    elif key == ord('['):
        # Decrease D0 down to a minimum of 5
        D0 = max(5, D0 - 5)
    elif key == ord(']'):
        # Increase D0 up to a maximum of 200
        D0 = min(200, D0 + 5)

    # --------- Adjust band width W ---------
    elif key == ord('w'):
        # Decrease band width W (not less than 5)
        band_W = max(5, band_W - 5)
    elif key == ord('W'):
        # Increase band width W (not more than 200)
        band_W = min(200, band_W + 5)

    # --------- Adjust Butterworth order ---------
    elif key == ord('1'):
        # Set Butterworth order to 1
        butter_n = 1
    elif key == ord('2'):
        # Set Butterworth order to 2
        butter_n = 2
    elif key == ord('3'):
        # Set Butterworth order to 3
        butter_n = 3

    # --------- Adjust kernel size for blur & Canny ---------
    elif key in [ord('+'), ord('=')]:
        # Increase kernel size by 2 if in blur or Canny mode
        if mode in ['canny', 'blur']:
            new_size = kernel_size[0] + 2
            # Limit maximum kernel size to 51
            if new_size <= 51:
                kernel_size = (new_size, new_size)
    elif key == ord('-'):
        # Decrease kernel size by 2 if in blur or Canny mode
        if mode in ['canny', 'blur']:
            new_size = kernel_size[0] - 2
            # Limit minimum kernel size to 1
            if new_size >= 1:
                kernel_size = (new_size, new_size)

    # --------- Toggle Canny AUTO / MANUAL thresholds ---------
    elif key == ord('a') or key == ord('A'):
        # Invert auto_mode flag
        auto_mode = not auto_mode

    # --------- Manual low threshold control ---------
    elif key in [ord('<'), ord(',')]:
        # Decrease low threshold by 10 in manual Canny mode
        if mode == 'canny' and not auto_mode:
            low_threshold = max(10, low_threshold - 10)
    elif key in [ord('>'), ord('.')]:
        # Increase low threshold by 10 in manual Canny mode
        if mode == 'canny' and not auto_mode:
            low_threshold = min(200, low_threshold + 10)

    # --------- Manual high threshold control ---------
    elif  key == ord('u'):
        # Increase high threshold by 20 (up to 255)
        if mode == 'canny' and not auto_mode:
            high_threshold = min(255, high_threshold + 20)
    elif  key == ord('d'):
        # Decrease high threshold by 20 (down to 40)
        if mode == 'canny' and not auto_mode:
            high_threshold = max(40, high_threshold - 20)


cap.release()

cv2.destroyAllWindows()


=== CONTROLS ===
c/b/s/n  - Canny/Blur/Sobel/Normal
l/h      - Low‑pass / High‑pass (frequency)
r/p      - BandReject / BandPass (frequency, uses W)
i/g/t    - Ideal / Gaussian / Butterworth filter type
[/]      - D0 (cutoff / band center) -/+
w/W      - Band width W -/+
1/2/3    - Butterworth order
+/-      - Kernel size for blur/Canny
A        - Toggle AUTO ↔ MANUAL thresholds (Canny)
</>      - Adjust LOW threshold (MANUAL only)
 u/d     - Adjust HIGH threshold ±20 (MANUAL only)
q        - Quit
